In [16]:
import os, tarfile
import re
import multiprocessing
import pandas as pd
import pyBigWig
import pybedtools
import zipfile, gzip
from gtfparse import read_gtf
import pandas as pd
#pd.set_option('display.max_columns', None)
from Bio.Seq import Seq

In [17]:
reference_genome_path = "/mnt/data05/shared/pdutta_data/Human_Genome_Data/UCSC/genome_by_sequence/"
data_path= "/mnt/data05/shared/pdutta_data/Human_Genome_Data/UCSC/gencode_annotation/"
donor_path = "/mnt/data05/shared/pdutta_data/Human_Genome_Data/Donor_splice_sites/"
non_donor_path = "/mnt/data05/shared/pdutta_data/Human_Genome_Data/Non_donnor_splice_sites/"
file_name="gencodeV38.bb"
gencode_path= "/mnt/data05/shared/pdutta_data/Human_Genome_Data/gencode_annotation/"
gtf_annotation_filename= "gencode.v38.annotation.gtf"
gff_annotation_filename= "gencode.v38.annotation.gff3"
file_path= data_path+file_name
gencode_gtf_file_path= gencode_path + gtf_annotation_filename
gencode_gff_file_path= gencode_path + gff_annotation_filename
#reference_genome_file_path = reference_genome_path + reference_genome_name
data_save_path= "/mnt/data05/shared/pdutta_data/Human_Genome_Data/region_wise_sequence/"

In [3]:
bb= pyBigWig.open(file_path)

In [4]:
df = read_gtf(gencode_gtf_file_path)

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


In [5]:
print(df.columns)
print(df.shape)

Index(['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand',
       'frame', 'gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id',
       'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name',
       'transcript_support_level', 'tag', 'havana_transcript', 'exon_number',
       'exon_id', 'ont', 'protein_id', 'ccdsid'],
      dtype='object')
(3150424, 25)


In [14]:
df.iloc[2:5]

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
2,chr1,HAVANA,exon,11869,12227,NaN,+,0,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,1,ENSE00002234944.1,,,
3,chr1,HAVANA,exon,12613,12721,NaN,+,0,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,2,ENSE00003582793.1,,,
4,chr1,HAVANA,exon,13221,14409,NaN,+,0,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,3,ENSE00002312635.1,,,


In [7]:
promoter_coordinates = [-249, 50]
core_promoter_coordinates= [-34, 35]
donor_coordinates= [-199, 200]
acceptor_coordinates= [-199, 200]

In [8]:
def remove_data(elements, distance):
    #elements = sorted(elements) # sorting the user input
    new_elements = [elements[0]] # make a new list for output
    for element in elements[1:]: # Iterate over the remaining elements...
        if element - new_elements[-1] >= distance: 
            # this is the condition you described above
            new_elements.append(element)
    return new_elements

In [9]:
def string_subs(row, temp_seq):
    start = row['coordinates'] -35
    end = row['coordinates'] + 34
    #print (start, end)
    sub_string = temp_seq[start:end]
    #print(sub_string)
    temp_seq = temp_seq.replace(sub_string, 70*'$')
    return temp_seq

In [15]:
chr_file= "chr4.fa"
reference_genome_file_path = reference_genome_path + chr_file
chr_name = chr_file.split('.')[0]
print (chr_name)
df_chr= df[df['seqname']==chr_name]
df_new = df_chr[df_chr['feature']=='exon'].reset_index(drop=True)
print (df_new.shape)
#print (reference_genome_file_path)
length= bb.chroms(chr_name)
print (length)
fasta = pybedtools.example_filename(reference_genome_file_path)
temp_seq= pybedtools.BedTool.seq(chr_name+':1-'+str(length), fasta)
print ("length:", len(temp_seq))

pattern_donor= ['GT', 'gt']

df_donor = pd.read_csv(donor_path+chr_name+"_donor_splice_set.csv")


print(df_donor.shape)



for index, row in df_donor.loc[20500:21500].iterrows():
    if (index%5==0):
        print (index)
    start = row['coordinates'] -35
    end = row['coordinates'] + 34
    #print (start, end)
    sub_string = temp_seq[start:end]
    #print(sub_string)
    temp_seq = temp_seq.replace(sub_string, 70*'$')


Non_donor_sequence = re.sub('[$]', '', temp_seq)
print ("@@##$$", len(Non_donor_sequence))
non_donor_indices = []
for j in pattern_acceptor:
    non_donor_indices.append([match.start() for match in re.finditer(j, Non_donor_sequence)])

print (len(non_donor_indices[0]))
flat_list = [item for sublist in non_donor_indices for item in sublist]
print ("test")
flat_list.sort()
print ("test1")
print ("@#@",len(flat_list))
removed_flat_list = remove_data(flat_list, 70)
print ("$$$", len(removed_flat_list))
print (removed_flat_list[0:15])

list_non_donor = []
for i in removed_flat_list[0:10]:
    temps =  Non_donor_sequence[i-35:i+34]
    list_non_donor.append(temps)
pd_df = pd.DataFrame(list_non_donor)
print (pd_df)
#pd_df.to_csv(non_acceptor_path+"/"+chr_name+"_non_acceptor.csv")


chr4
(62342, 25)
190214555
length: 190214555
(53730, 6)
20500
20505
20510
20515
20520
20525
20530
20535
20540
20545
20550
20555
20560
20565
20570
20575
20580
20585
20590
20595
20600
20605
20610
20615
20620
20625
20630
20635
20640
20645
20650
20655
20660
20665
20670
20675
20680
20685
20690
20695
20700
20705
20710
20715
20720
20725
20730
20735
20740
20745
20750
20755
20760
20765
20770
20775
20780
20785
20790
20795
20800
20805
20810
20815
20820
20825
20830
20835
20840
20845
20850
20855
20860
20865
20870
20875
20880
20885
20890
20895
20900
20905
20910
20915
20920
20925
20930
20935
20940
20945
20950
20955
20960
20965
20970
20975
20980
20985
20990
20995
21000
21005
21010
21015
21020
21025
21030
21035
21040
21045
21050
21055
21060
21065
21070
21075
21080
21085
21090
21095
21100
21105
21110
21115
21120
21125
21130
21135
21140
21145
21150
21155
21160
21165
21170
21175
21180
21185
21190
21195
21200
21205
21210
21215
21220
21225
21230
21235
21240
21245
21250
21255
21260
21265
21270
21275
21280
21

KeyboardInterrupt: 